In [5]:
from dotenv import load_dotenv
load_dotenv()
from langchain_cohere import ChatCohere

In [6]:
model = ChatCohere(model="command-r")

In [ ]:
model.invoke("how can langsmith help with testing?")

# Simple prompt

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following sentences into Japanese Romanized"),
    HumanMessage("Kento Nanami is a nice person.")
]

ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Nanami Kento wa ii hito da.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '3c5f5623-96cd-43bf-9727-8454a8d72ced', 'token_count': {'input_tokens': 85, 'output_tokens': 9}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '3c5f5623-96cd-43bf-9727-8454a8d72ced', 'token_count': {'input_tokens': 85, 'output_tokens': 9}}, id='run-18948152-43a1-4057-8546-07d49cd16252-0')

# Using output parsers

In [10]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

chain.invoke(messages)

'Nanami Kento wa ii hito da.'

# Using prompt templates

In [7]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Translate the following into {language}"),
    ("user", "{text}")
])

prompt = prompt_template.invoke({
    "language": "Spanish",
    "text": "Kento Nanami is a nice person",
})
prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following into Spanish'), HumanMessage(content='Kento Nanami is a nice person')])

<script src="https://kit.fontawesome.com/28934d071c.js" crossorigin="anonymous"></script>
<i class="fas fa-sticky-note"></i> Note that the returned value is a `ChatPromptValue` object but models accept messages as well as ChatPromptValue as input

In [13]:
chain = prompt_template | model | parser

chain.invoke({
    "language": "Hinglish",
    "text": "Kento Nanami is a nice person"
})

'Kento Nanami hai ek accha aadmi.'

# LangServe
Refer the main.py file for LangServe implementation

# Using Message Place Holders

In [8]:
from typing import List, Union

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

from langserve.pydantic_v1 import BaseModel, Field

In [9]:
class InputChat(BaseModel):
    """Input for the chat endpoints."""

    messages: List[Union[SystemMessage, HumanMessage, AIMessage]] = Field(
        ...,
        description = "The chat messages representing the current conversation."
    )

In [10]:
parser = StrOutputParser()

system_template = "You are a helpful, professional assistant named Cob."
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt_template | model | parser

In [15]:
chats = InputChat(messages=[HumanMessage("Hey, what time are you free?")])

In [18]:
chain = model | parser
res = chain.invoke(
    [
        HumanMessage("Hi! I'm Mark."),
        AIMessage("What's up loser"),
        HumanMessage("Why are you being rude to me?")
    ]
)
res

"I apologize for being rude. I was just joking. How's it going, Mark? I hope you're doing well today! Are you up to anything exciting?"